In [139]:
#naming rows and columns
rows = 'ABCDEFGHI'
cols = '123456789'

In [140]:
def cross(a, b):
      return [s+t for s in a for t in b]

In [142]:
#creating out sudoku box
boxes = cross(rows, cols)

In [143]:
row_units = [cross(r, cols) for r in rows]
# Element example:
# row_units[0] = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
# This is the top most row.

column_units = [cross(rows, c) for c in cols]
# Element example:
# column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
# This is the left most column.

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
# Element example:
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']
# This is the top left square.

unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)


In [146]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [152]:
def grid_values_old(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    assert len(grid) == 81, "Input grid must be a string of length 81 (9x9)"
    return dict(zip(boxes, grid))

In [217]:
grid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
grid2 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid3= '8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..' #worlds hardest sudoku

In [220]:
display(grid_values_old(grid))

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 


In [221]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

In [222]:
display(grid_values(grid))

    4     123456789 123456789 |123456789 123456789 123456789 |    8     123456789     5     
123456789     3     123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |    7     123456789 123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     2     123456789 |123456789 123456789 123456789 |123456789     6     123456789 
123456789 123456789 123456789 |123456789     8     123456789 |    4     123456789 123456789 
123456789 123456789 123456789 |123456789     1     123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |    6     123456789     3     |123456789     7     123456789 
    5     123456789 123456789 |    2     123456789 123456789 |123456789 123456789 123456789 
    1     123456789     4     |123456789 123456789 123456789 |12345678

In [196]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values


In [197]:
display(eliminate(grid_values(grid)))

   4      1679   12679  |  139     2369    1269  |   8      1239     5    
 26789     3    1256789 | 14589   24569  1245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569 1245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     135789 |  3459   34579    4579  | 13579     6     13789  
  3679   15679   135679 |  359      8     25679  |   4     12359   12379  
 36789   456789  356789 |  3459     1     245679 | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789   36789  |   2      479    14789  |  1369   13489   134689 
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


In [223]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [224]:
display(only_choice(eliminate(grid_values(grid))))

   4      1679   12679  |  139     2369    1269  |   8      1239     5    
 26789     3    1256789 | 14589   24569  1245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569 1245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     135789 |  3459   34579    4579  | 13579     6     13789  
  3679   15679   135679 |  359      8     25679  |   4     12359   12379  
 36789     4     356789 |  3459     1     245679 | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |  1369   13489   134689 
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


In [203]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        # Use the Eliminate Strategy
        values = eliminate(values)
        # Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [215]:
display(reduce_puzzle(grid_values(grid2)))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


In [209]:
def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
#     return n,s
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [218]:
display(search(grid_values(grid3)))

8 1 2 |7 5 3 |6 4 9 
9 4 3 |6 8 2 |1 7 5 
6 7 5 |4 9 1 |2 8 3 
------+------+------
1 5 4 |2 3 7 |8 9 6 
3 6 9 |8 4 5 |7 2 1 
2 8 7 |1 6 9 |5 3 4 
------+------+------
5 2 1 |9 7 4 |3 6 8 
4 3 8 |5 2 6 |9 1 7 
7 9 6 |3 1 8 |4 5 2 
